In [1]:
!pip install pandas
!pip install tqdm
!pip install sklearn
!pip install gensim==4.2.0
!pip install Cython

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [3]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import gensim
import Cython

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
!nvidia-smi

Fri Oct  7 15:25:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:81:00.0 Off |                  N/A |
| 30%   32C    P8    16W / 350W |      3MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
raw = json.load(open(r'data/backup/train.json'))

In [6]:
labels = []
data = []
d2v = gensim.models.doc2vec.Doc2Vec.load(r'doc2vec.model')

for i, record in enumerate(raw):
    if i%1000 == 0:
        print(i)
        
    proauthors = [i for i in record['authors'] if i < 100]
    label = np.zeros(101)
    # label[proauthors] = 1.
    if len(proauthors) > 0:
        label[proauthors] = 1.
    else: 
        label[-1] = 1.
    labels.append(label)
    
    text = [str(i) for i in record['title']]
    text.extend([str(i) for i in record['abstract']])
    text = d2v.infer_vector(text)
    
    coauthors = np.zeros(21146)
    coauthors[[i-100 for i in [i for i in record['authors'] if i >= 100]]] = 1.
    
    venue = np.zeros(465)
    venue[[record['venue']] if record['venue'] != '' else []] = 1.
    
    data.append(np.concatenate([text, coauthors, venue], axis=0))
    
labels = np.stack(labels, axis=0)
data = np.stack(data, axis=0)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000


In [7]:
ids = np.where(labels[:, -1]==0)
data_t = data[ids]
labels_t = labels[ids]

ids = np.where(labels[:, -1]==1)
data_f = data[ids]
labels_f = labels[ids]

data_t, labels_t = resample(data_t, labels_t, replace=True, n_samples=int(labels_f.shape[0]), random_state=51)

data = np.concatenate([data_t, data_f])
labels = np.concatenate([labels_t, labels_f])


In [8]:
data.shape

(36666, 21867)

In [9]:
labels.shape

(36666, 101)

In [10]:
data_train, data_valid, labels_train, labels_valid = train_test_split(data, labels, test_size=0.2, random_state=51)

In [11]:
train_set = TensorDataset(torch.tensor(data_train, dtype=torch.float), torch.tensor(labels_train, dtype=torch.float))
valid_set = TensorDataset(torch.tensor(data_valid, dtype=torch.float), torch.tensor(labels_valid, dtype=torch.float))

In [12]:
class AuthorAttriClf(nn.Module):
    def __init__(self):
        super(AuthorAttriClf, self).__init__()
        
        self.clf_block = nn.Sequential(
            nn.Linear(21867, 2048),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(1024, 101),
        )

    def forward(self, input):
        probs = self.clf_block(input)

        return probs    

In [13]:
def train(train_status, model, optim, scheduler, criterion, epoch_size, train_loader, valid_loader):
   
    for epoch in range(epoch_size):
        model.train()
        epoch_loss = 0
        epoch_labels = torch.Tensor([])
        epoch_preds = torch.Tensor([])

        train_loop = tqdm(enumerate(train_loader), total=len(train_loader))
        train_loop.set_description(f"Epoch [{epoch+1}/{epoch_size}]")

        for batch, (inputs, labels) in train_loop:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optim.zero_grad()
            loss.backward()
            optim.step() 

            epoch_loss += loss.item()
            epoch_preds = torch.cat(((epoch_preds, (outputs.detach().cpu() > 0.5).int())), 0)
            epoch_labels = torch.cat((epoch_labels, labels.detach().cpu()), 0)

            train_loop.set_postfix_str(
                'train_loss={:.5f}'.format(loss.item())
            )

            if batch == len(train_loader)-1:
                epoch_loss /= len(train_loader.dataset)/train_loader.batch_size
                train_f1 = f1_score(epoch_labels, epoch_preds, average='samples', zero_division=1)
                valid_f1 = validate(model, valid_loader)
                train_loop.set_postfix_str(
                    'train_loss={:.5f}, train_f1={:.5f}, valid_f1={:.5f}'.format(
                        epoch_loss, train_f1, valid_f1
                    )
                )

                scheduler.step()

def validate(model, valid_loader):
    model.eval()
    valid_labels = torch.Tensor([])
    valid_preds = torch.Tensor([])
    with torch.no_grad():
        for batch, (inputs, labels) in enumerate(valid_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)

            valid_preds = torch.cat(((valid_preds, (outputs.detach().cpu() > 0.5).int())), 0)
            valid_labels = torch.cat((valid_labels, labels.detach().cpu()), 0)

    return f1_score(valid_labels, valid_preds, average='samples', zero_division=1)
    

In [14]:
epoch_size = 30
batch_size = 128
lr = 1e-3

model = AuthorAttriClf().to(device)
optim = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optim, lr_lambda=lambda epoch: 0.96)
criterion = nn.BCEWithLogitsLoss()

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)

train_status = {'train_loss': []}
train(train_status, model, optim, scheduler, criterion, epoch_size, train_loader, valid_loader)

Epoch [30/30]: 100%|██████████| 230/230 [00:04<00:00, 47.95it/s, train_loss=0.00122, train_f1=0.97277, valid_f1=0.89365]


In [15]:
df_test = pd.read_json(r'data/backup/test.json')

pred_ids = []
for i in df_test['identifier']:
    pred_ids.append(i)
pred_ids = np.array(pred_ids)

data_test = []
for _, row in df_test.iterrows():
    text = [str(i) for i in row['title']]
    text.extend([str(i) for i in row['abstract']])
    text = d2v.infer_vector(text)
    
    coauthors = np.zeros(21146)
    coauthors[[i-100 for i in [i for i in row['coauthors'] if i >= 100]]] = 1.
    
    venue = np.zeros(465)
    venue[[row['venue']] if row['venue'] != '' else []] = 1.
    
    data_test.append(np.concatenate([text, coauthors, venue], axis=0))
    
data_test = np.stack(data_test)

test_set = TensorDataset(torch.tensor(data_test, dtype=torch.float), torch.tensor(pred_ids, dtype=torch.float))
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=0)

In [16]:
def get_predictions(model, test_loader):
    with torch.no_grad():
        preds = []
        identifiers = []

        for batch, (inputs, ids) in enumerate(test_loader):
            inputs = inputs.to(device)
            outputs = torch.sigmoid(model(inputs))

            for i in range(outputs.shape[0]):
                identifiers.append(int(ids[i]))
                pred = torch.nonzero((outputs[i].cpu() > 0.5)[:-1].int())
                if len(pred) > 0:
                    preds.append(" ".join([str(int(i)) for i in pred]))
                else:
                    preds.append("-1")
        df = pd.DataFrame({'ID': identifiers, 'Predict': preds})
        df.to_csv(r'data/pred.csv', sep=',', index=False, encoding='utf-8')

get_predictions(model, test_loader)